In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

In [2]:
def get_text_list(file_path):
    with open(file_path, 'rb') as f:
        texts = pd.read_csv(f, sep='\t', encoding='utf-16 LE')
    return list(texts[texts['text_graph_desc'] == True]['text_id'])

In [3]:
paths = ['/home/zlovoblachko/local_realec/Exam2014/Task 1/Exam2014_text_table.tsv',
         '/home/zlovoblachko/local_realec/Exam2015/Task 1/Exam2015_text_table.tsv',
         '/home/zlovoblachko/local_realec/Exam2016/Task 1/Exam2016_text_table.tsv',
         '/home/zlovoblachko/local_realec/Exam2017/Task 1/Exam2017_text_table.tsv',
         '/home/zlovoblachko/local_realec/Exam2019/Task 1/Exam2019_text_table.tsv',
         '/home/zlovoblachko/local_realec/Exam2020/Task 1/Exam2020_text_table.tsv',
         '/home/zlovoblachko/local_realec/Exam2020/Task 2/Exam2020_text_table.tsv',
         '/home/zlovoblachko/local_realec/Exam2020/Task 3/Exam2020_text_table.tsv',
         '/home/zlovoblachko/local_realec/Exam2020/Task 4/Exam2020_text_table.tsv',
         '/home/zlovoblachko/local_realec/Exam2020/Task 5/Exam2020_text_table.tsv',
         '/home/zlovoblachko/local_realec/Exam2020/Task 6/Exam2020_text_table.tsv',
         '/home/zlovoblachko/local_realec/Exam2020/Task 7/Exam2020_text_table.tsv',
         '/home/zlovoblachko/local_realec/Exam2020/Task 8/Exam2020_text_table.tsv']

relevant_texts_nums = []

for path in paths:
    relevant_texts_nums.append(get_text_list(path))

In [10]:
def get_sentences_tables(file_path, texts_list):
    with open(file_path, 'rb') as f:
        year_corpus = pd.read_csv(f, sep='\t', encoding='utf-16 LE')
    year_corpus['year'] = '_'.join(file_path.split('/')[5:7])
    return year_corpus[year_corpus['text_id'].isin(texts_list)]

In [11]:
paths_sentences = ['/home/zlovoblachko/diploma/data/Exam2014/Task 1/exam2014_new_sentence_text_table.tsv',
                  '/home/zlovoblachko/diploma/data/Exam2015/Task 1/Exam2015_new_sentence_text_table.tsv',
                  '/home/zlovoblachko/diploma/data/Exam2016/Task 1/exam2016_new_sentence_text_table.tsv',
                  '/home/zlovoblachko/diploma/data/Exam2017/Task 1/Exam2017_new_sentence_text_table.tsv',
                  '/home/zlovoblachko/diploma/data/Exam2019/Task 1/exam2019_new_sentence_text_table.tsv',
                  '/home/zlovoblachko/diploma/data/Exam2020/Task 1/Exam2020_new_sentence_text_table.tsv',
                  '/home/zlovoblachko/diploma/data/Exam2020/Task 2/Exam2020_new_sentence_text_table.tsv',
                  '/home/zlovoblachko/diploma/data/Exam2020/Task 3/Exam2020_new_sentence_text_table.tsv',
                  '/home/zlovoblachko/diploma/data/Exam2020/Task 4/Exam2020_new_sentence_text_table.tsv',
                  '/home/zlovoblachko/diploma/data/Exam2020/Task 5/Exam2020_new_sentence_text_table.tsv',
                  '/home/zlovoblachko/diploma/data/Exam2020/Task 6/Exam2020_new_sentence_text_table.tsv',
                  '/home/zlovoblachko/diploma/data/Exam2020/Task 7/Exam2020_new_sentence_text_table.tsv',
                  '/home/zlovoblachko/diploma/data/Exam2020/Task 8/Exam2020_new_sentence_text_table.tsv']

all_sentences_df = pd.DataFrame()

In [12]:
path_mistakes = "/home/zlovoblachko/diploma/Labelled_dataset.tsv"

In [13]:
for file_path, text_list in zip(paths_sentences, relevant_texts_nums):
    all_sentences_df = pd.concat([all_sentences_df, get_sentences_tables(file_path, text_list)])

all_sentences_df.head()

,text_id,paragraph_id,sentence_id,sentence_tokens,sentence_poses,sentence_lemmas,sentence_token_spaces,sentence_orig_tokens,sentence_orig_token_spaces,sentence_token_deps,sentence_token_heads,sentence_token_spacy_poses,sentence_token_spacy_tags,year
0,1,1,1,Given diagram visualize the proportion of popu...,AJ0 NN1 VVB AT0 NN1 PRF NN1 PRP CRD CJC AVP PR...,given|give diagram visualize the proportion of...,111111111111011111111101,NaN,NaN,prep pobj ROOT det dobj prep pobj amod npadvmo...,2 0 2 4 2 4 5 6 7 8 8 4 11 12 12 14 18 18 14 4...,VERB NOUN VERB DET NOUN ADP NOUN VERB NUM CCON...,"VBN NN VB DT NN IN NN VBN CD CC RB IN NNP , NN...",Exam2014_Task 1
1,1,2,2,"In all of the countries , the proportion was g...",PRP DT0 PRF AT0 NN2 PUN AT0 NN1 VBD VVG AV0 PR...,"in all of the country , the proportion be grow...",111101111111101111101111111101111011101101,NaN,NaN,prep pobj prep det pobj punct det nsubj aux RO...,9 0 1 4 2 9 7 9 9 9 9 9 13 11 9 9 17 15 19 17 ...,ADP PRON ADP DET NOUN PUNCT DET NOUN AUX VERB ...,"IN DT IN DT NNS , DT NN VBD VBG RB IN DT NN , ...",Exam2014_Task 1
2,1,3,3,"After a stable period , we see a huge incline ...",PRP AT0 AJ0 NN1 PUN PNP VVB AT0 AJ0 NN1 CJT VM...,"after a stable period , we see a huge incline ...",11101111111111111101,NaN,NaN,prep det amod pobj punct nsubj ROOT det amod d...,6 3 3 0 6 6 6 9 9 6 12 12 9 12 13 12 15 15 17 6,ADP DET ADJ NOUN PUNCT PRON VERB DET ADJ NOUN ...,"IN DT JJ NN , PRP VBP DT JJ NN WDT MD VB IN NN...",Exam2014_Task 1
3,1,3,4,About a third of a population would be aged 65...,PRP AT0 ORD PRF AT0 NN1 VM0 VBI PRP CRD CJC AV...,about a third of a population would be aged 65...,1111111111111011111011101,NaN,NaN,advmod quantmod nsubj prep det pobj aux ROOT a...,2 2 7 2 5 3 7 7 7 8 9 9 7 12 7 7 15 15 19 20 2...,ADV DET NOUN ADP DET NOUN AUX AUX VERB NUM CCO...,"RB DT NN IN DT NN MD VB VBN CD CC RB IN CD , V...",Exam2014_Task 1
4,1,4,5,"USA , however , had bigger part of old people ...",NP0 PUN AV0 PUN VHD AJC NN1 PRF AJ0 NN0 PRP AT...,"usa , however , have big part of old people th...",0101111111110110111110110111111111101101111101,"USA , however , had bigger part of old people ...",0101111111110110111110110111111111101101111101,nsubj punct advmod punct ROOT amod dobj prep a...,4 4 4 4 4 6 4 6 9 7 4 12 10 4 4 16 14 14 17 14...,PROPN PUNCT ADV PUNCT VERB ADJ NOUN ADP ADJ NO...,"NNP , RB , VBD JJR NN IN JJ NNS IN DT NN , VBG...",Exam2014_Task 1


In [15]:
with open(path_mistakes, "rb") as f:
    mistakes_df = pd.read_csv(f, sep='\t')

In [16]:
mistakes_df.head()

,level_0,index,mistake_id,text_id,sentence_id,mistake_type,error_span,correction,span_start,span_end,year,bigger_code,first_level_tag,second_level_tag
0,4,7,8,1,2,Absence_explanation,century,XXth century,200,207,Exam2014_Task 1,Discourse,M,MULTIWORD
1,6,10,11,1,4,Absence_explanation,a population,the Japanese population,436,448,Exam2014_Task 1,Discourse,R,MULTIWORD
2,7,11,12,1,4,Tense_choice,would be,is going to be,449,457,Exam2014_Task 1,Grammar,R,VERB
3,8,12,13,1,4,Punctuation,",",-,482,483,Exam2014_Task 1,Punctuation,R,PUNCT
4,10,16,17,1,5,Absence_explanation,century,XXth century,583,590,Exam2014_Task 1,Discourse,M,MULTIWORD


In [17]:
all_sentences_filtered_df = all_sentences_df[['text_id', 'paragraph_id', 'sentence_id', 'sentence_tokens', 'year']]
all_sentences_filtered_df.head()

,text_id,paragraph_id,sentence_id,sentence_tokens,year
0,1,1,1,Given diagram visualize the proportion of popu...,Exam2014_Task 1
1,1,2,2,"In all of the countries , the proportion was g...",Exam2014_Task 1
2,1,3,3,"After a stable period , we see a huge incline ...",Exam2014_Task 1
3,1,3,4,About a third of a population would be aged 65...,Exam2014_Task 1
4,1,4,5,"USA , however , had bigger part of old people ...",Exam2014_Task 1


In [28]:
all_sentences_filtered_df.sort_values(by=["year", "text_id", "sentence_id"], inplace=True)
mistakes_df.sort_values(by=["year", "text_id", "sentence_id"], inplace=True)

/tmp/ipykernel_1675/3529073697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sentences_filtered_df.sort_values(by=["year", "text_id", "sentence_id"], inplace=True)


In [29]:
training_data = []

In [30]:
for _, sentence_row in all_sentences_filtered_df.iterrows():
    text_id = sentence_row["text_id"]
    sentence_id = sentence_row["sentence_id"]
    sentence = sentence_row["sentence_tokens"]
    mistakes_in_sentence = mistakes_df[(mistakes_df["text_id"] == text_id) & (mistakes_df["sentence_id"] == sentence_id)]
    annotations = []
    for _, mistake_row in mistakes_in_sentence.iterrows():
        span_start = mistake_row["span_start"]
        span_end = mistake_row["span_end"]
        annotations.append({
            "start": span_start,
            "end": span_end,
            "first_level_tag": str(mistake_row["first_level_tag"]).strip(),
            "second_level_tag": str(mistake_row["second_level_tag"]).strip(),
            "correction": mistake_row["correction"]
        })
    training_data.append({"text": sentence, "annotations": annotations})

In [31]:
output_filename = "Final_training_data.jsonl"

with open(output_filename, "w", encoding="utf-8") as f:
    for entry in training_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"Generated {output_filename} with {len(training_data)} entries.")

Generated Final_training_data.jsonl with 85066 entries.


## SpaCy

In [36]:
import spacy
from spacy.tokens import DocBin
import random

In [33]:
nlp = spacy.blank("en")
jsonl_file = "Final_training_data.jsonl"
output_spacy_file = "Final_training_data.spacy"

In [34]:
with open(jsonl_file, "r", encoding="utf-8") as f:
    training_data = [json.loads(line) for line in f]

doc_bin = DocBin()

In [35]:
for entry in training_data:
    text = entry["text"]
    annotations = entry["annotations"]
    doc = nlp.make_doc(text)
    spans = []
    for ann in annotations:
        start, end = ann["start"], ann["end"]
        first_level_tag = ann["first_level_tag"]
        second_level_tag = ann["second_level_tag"]
        label = f"{first_level_tag}-{second_level_tag}"
        span = doc.char_span(start, end, label=label)
        if span:
            spans.append(span)
    doc.spans["sc"] = spans
    doc_bin.add(doc)
doc_bin.to_disk(output_spacy_file)

In [37]:
nlp = spacy.blank("en")
doc_bin = DocBin().from_disk(output_spacy_file)
docs = list(doc_bin.get_docs(nlp.vocab))

In [38]:
random.seed(42)
random.shuffle(docs)

In [39]:
train_ratio = 0.7
dev_ratio = 0.2
test_ratio = 0.1

In [40]:
total_docs = len(docs)
train_end = int(total_docs * train_ratio)
dev_end = train_end + int(total_docs * dev_ratio)

In [41]:
train_docs = docs[:train_end]
dev_docs = docs[train_end:dev_end]
test_docs = docs[dev_end:]

In [42]:
output_files = {
    "train.spacy": train_docs,
    "dev.spacy": dev_docs,
    "test.spacy": test_docs
}

In [43]:
for filename, doc_subset in output_files.items():
    doc_bin_subset = DocBin()
    for doc in doc_subset:
        doc_bin_subset.add(doc)
    doc_bin_subset.to_disk(filename)